In [ ]:
# load the model
# load audio data
# pass through audio tokenizer, prepare tokens according to the task
# pass through model
# post process outputs based on the task
# return text/audio tokens

In [ ]:
import torch
from datasets import load_dataset, Audio
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers import Pipeline, pipeline
from transformers.pipelines import PIPELINE_REGISTRY

In [ ]:
ds = load_dataset("PolyAI/minds14", name="en-US", split="train")

In [ ]:
import re
import numpy as np
from transformers import MimiModel, GenerationConfig

In [ ]:
class IndriPipeline(Pipeline):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.audio_tokenizer = MimiModel.from_pretrained('kyutai/mimi').to(device=self.device)

        # Ideally all of this should come from model config
        self.convert_token = self.tokenizer.encode('[convert]')
        self.stop_token = self.tokenizer.encode('[stop]')
        self.text_modality_token = self.tokenizer.encode('[text]')
        self.acoustic_modality_token = self.tokenizer.encode('[mimi]')
        self.num_codebooks = 8
        self.audio_offset = 50257

        self.model.generation_config = GenerationConfig(
            eos_token_id=self.stop_token,
            max_length=kwargs.get('max_length', 1024),
            temperature=kwargs.get('temperature', 0.5),
            top_k=kwargs.get('top_k', 15),
            do_sample=kwargs.get('do_sample', True)
        )

    def _sanitize_parameters(self, **kwargs):
        task = kwargs.get('task', 'tts')
        assert task in ['tts', 'asr'], f'Task must be one of tts, asr. You provided: {task}'

        speaker = kwargs.get('speaker', '[spkr_unk]')

        preprocess_kwargs = {
            'task': task,
            'speaker': speaker
        }

        return preprocess_kwargs, {}, {}

    def _prepare_tts_tokens(self, text_tokens, speaker):
        input_tokens = np.hstack([
            self.text_modality_token,
            text_tokens,
            self.convert_token,
            self.acoustic_modality_token,
            self.tokenizer.encode(speaker)
        ])

        return input_tokens.tolist()

    def _prepare_asr_tokens(self, audio_tokens):
        pass

    def _sanitize_text(self, text):
        text = text.lower()
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'[ \t]+', ' ', text)

        text = re.sub(r'([,\.?])+', r'\1', text)

        return text.strip()

    def _deserialize_tokens(self, tokens):
        tokens = tokens.view(self.num_codebooks, -1)
        min_shape = torch.min(tokens.shape[1], dim=1).values
        acoustic_tokens = tokens[:, :min_shape] - 2048 * torch.arange(num_codebooks, device=tokens.device).unsqueeze(1)

        return acoustic_tokens

    def preprocess(self, inputs, speaker, task):
        if task == 'tts':
            input_text = self._sanitize_text(inputs)
            input_tokens = self.tokenizer.encode(inputs)
            task_tokens = self._prepare_tts_tokens(input_tokens, speaker)
            task_tokens = torch.tensor(task_tokens).unsqueeze(0)

        elif task == 'asr':
            raise ValueError('ASR task is not yet supported')

        return {'task_tokens': task_tokens}

    def _forward(self, model_inputs, **forward_args):

        outputs = self.model.generate(model_inputs['task_tokens'])
        outputs = outputs[:, model_inputs['task_tokens'].shape[-1]:]

        end = torch.where(outputs == self.stop_token[0])[-1]

        if end.shape[-1] > 0:
            end = end[0]
        else:
            end = outputs.shape[-1]

        outputs = outputs[:, :end]
        outputs -= self.audio_offset

        # audio_tokens = self._deserialize_tokens(outputs)
        # print(audio_tokens.shape, audio_tokens)
        # audio = self.audio_tokenizer.decode(audio_tokens).audio_values

        return {
            'model_output': outputs,
            # 'audio': audio
        }

    def postprocess(self, model_outputs):
        return model_outputs

In [ ]:
PIPELINE_REGISTRY.register_pipeline(
    "indri",
    pipeline_class=IndriPipeline,
    pt_model=AutoModelForCausalLM
)

In [ ]:
pipe = pipeline("indri", model="cmeraki/mimi_124m_8cb", device=torch.device('cuda:0'))

In [ ]:
o = pipe(["my name is romit"])

In [ ]:
toks = o[0]['model_output'][0]

In [ ]:
toks.shape

In [ ]:
toks.view(8, -1)

In [ ]:
def _deserialize_tokens(tokens):
    tokens = tokens.view(8, -1)
    min_shape = torch.min(tokens.shape[1], dim=1).values
    acoustic_tokens = tokens[:, :min_shape] - 2048 * torch.arange(num_codebooks, device=tokens.device).unsqueeze(1)

    return acoustic_tokens